In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

IDIR = 'data/'

priors_sampling_frac = 0.1

# Obtaining knowledge from prior orders

In [2]:
print('loading prior')
priors = pd.read_csv(IDIR + 'order_products__prior.csv',
                     dtype={
                         'order_id': np.int32,
                         'product_id': np.uint16,
                         'add_to_cart_order': np.int16,
                         'reordered': np.int8}).sample(frac=priors_sampling_frac)
print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))

print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv',
                     dtype={
                         'order_id': np.int32,
                         'user_id': np.int32,
                         'eval_set': 'category',
                         'order_number': np.int16,
                         'order_dow': np.int8,
                         'order_hour_of_day': np.int8,
                         'days_since_prior_order': np.float32})
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))

print('loading products')
products = pd.read_csv(IDIR + 'products.csv',
                       dtype={
                           'product_id': np.uint16,
                           'order_id': np.int32,
                           'aisle_id': np.uint8,
                           'department_id': np.uint8},
                       usecols=['product_id', 'aisle_id', 'department_id'])
print('products {}: {}'.format(products.shape, ', '.join(products.columns)))

loading prior
priors (3243449, 4): order_id, product_id, add_to_cart_order, reordered
loading orders
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
loading products
products (49688, 3): product_id, aisle_id, department_id


In [3]:
priors_orders = pd.merge(orders[orders['eval_set'] == 'prior'], priors, on='order_id')

print(priors_orders.shape)

del priors

(3243449, 10)


In [4]:
priors_orders.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2398795,1,prior,2,3,7,15.0,13176,4,0
2,473747,1,prior,3,3,12,21.0,12427,2,1
3,3108588,1,prior,8,1,14,14.0,12427,1,1
4,3108588,1,prior,8,1,14,14.0,46149,5,0
5,2295261,1,prior,9,1,16,0.0,196,4,1
6,2168274,2,prior,1,2,11,NaN,45066,13,0
7,2168274,2,prior,1,2,11,NaN,20574,3,0
8,2168274,2,prior,1,2,11,NaN,27344,9,0
9,738281,2,prior,4,2,10,8.0,36735,5,0


## Features by dataset

For auxiliary purposes

[x] Total number of users<br>
[x] Total number of orders<br>
[x] Total number of products

In [5]:
total_users = priors_orders['user_id'].nunique()
total_orders = priors_orders['order_id'].nunique()
total_products = priors_orders['product_id'].nunique()

## Features by user

[x]  Total number of orders<br>
[x]  Total number of products<br>
[x]  Total number of distinct products<br>
[x]  Total number of reordered products<br>
[x]  Average days since prior order<br>
[x]  Average number of products by order<br>
[x]  Average number of distinct products by order<br>
[x]  Average number of reordered products by order<br>
[x]  Rate of distinct products<br>
[x]  Rate of reordered products<br>

In [6]:
agg_func = {'order_id': ['nunique'],
            'product_id': ['count', 'nunique'],
            'reordered': ['sum'],
            'days_since_prior_order': ['mean']}

user_feats = priors_orders.groupby('user_id').agg(agg_func)

In [7]:
# renaming columns
user_feats = user_feats[['product_id', 'reordered', 'order_id', 'days_since_prior_order']]

user_feats.columns = ['_user_total_products', '_user_total_products_distinct',
                      '_user_total_products_reordered', '_user_total_orders',
                      '_user_avg_days_since_prior_order']

In [8]:
user_feats['_user_avg_products_by_order'] = (user_feats['_user_total_products'] /
                                             user_feats['_user_total_orders']).astype(np.float32)

user_feats['_user_avg_products_distinct_by_order'] = (user_feats['_user_total_products_distinct'] /
                                                      user_feats['_user_total_orders']).astype(np.float32)

user_feats['_user_avg_products_reordered_by_order'] = (user_feats['_user_total_products_reordered'] /
                                                       user_feats['_user_total_orders']).astype(np.float32)

In [9]:
user_feats['_user_rate_products_distinct'] = (user_feats['_user_total_products_distinct'] /
                                              user_feats['_user_total_products']).astype(np.float32)

user_feats['_user_rate_products_reordered'] = (user_feats['_user_total_products_reordered'] /
                                               user_feats['_user_total_products']).astype(np.float32)

In [10]:
user_feats['_user_total_products_reordered'] = user_feats['_user_total_products_reordered'].astype(np.int64)

In [11]:
# resetting index

# XXX testing
# user_feats = user_feats.reset_index()

In [12]:
# changing types

# XXX testing
# feat_dtype = {'user_id': np.int32,
feat_dtype = {'_user_total_products': np.uint16,
              '_user_total_products_distinct': np.uint16,
              '_user_total_products_reordered': np.uint16,
              '_user_total_orders': np.uint16,
              '_user_avg_days_since_prior_order': np.float32}

for (feat, dtype) in feat_dtype.items():
    user_feats[feat] = user_feats[feat].astype(dtype)

In [13]:
user_feats.head()

,_user_total_products,_user_total_products_distinct,_user_total_products_reordered,_user_total_orders,_user_avg_days_since_prior_order,_user_avg_products_by_order,_user_avg_products_distinct_by_order,_user_avg_products_reordered_by_order,_user_rate_products_distinct,_user_rate_products_reordered
user_id,,,,,,,,,,
1,6,4,3,5,12.800000,1.200000,0.800000,0.600000,0.666667,0.500000
2,22,20,11,10,15.894737,2.200000,2.000000,1.100000,0.909091,0.500000
3,9,7,5,7,13.333333,1.285714,1.000000,0.714286,0.777778,0.555556
4,4,4,0,3,14.000000,1.333333,1.333333,0.000000,1.000000,0.000000
5,4,4,3,3,14.750000,1.333333,1.333333,1.000000,1.000000,0.750000


## Features by product

[x]  Total number of orders<br>
[x]  Total number of users<br>
[x]  Total number of reorder<br>
[x]  Average days since prior order<br>
[x]  Rate of orders<br>
[x]  Rate of users<br>
[x]  Rate of reordered<br>

In [14]:
agg_func = {'order_id': ['nunique'],
            'user_id': ['nunique'],
            'reordered': ['sum'],
            'days_since_prior_order': ['mean']}

product_feats = priors_orders.groupby('product_id').agg(agg_func)

In [15]:
# renaming columns
product_feats = product_feats[['order_id', 'user_id', 'reordered', 'days_since_prior_order']]

product_feats.columns = ['_prod_total_orders', '_prod_total_users',
                         '_prod_total_reordered', '_prod_avg_days_since_prior_order']

In [16]:
product_feats['_prod_rate_orders'] = (product_feats['_prod_total_orders'] /
                                      total_orders).astype(np.float32)

product_feats['_prod_rate_users'] = (product_feats['_prod_total_users'] /
                                     total_users).astype(np.float32)

product_feats['_prod_rate_reordered'] = (product_feats['_prod_total_reordered'] /
                                         product_feats['_prod_total_orders']).astype(np.float32)

In [20]:
# resetting index
product_feats = product_feats.reset_index()

In [21]:
# changing types
feat_dtype = {'product_id': np.uint16,
              '_prod_total_orders': np.uint16,
              '_prod_total_users': np.uint16,
              '_prod_total_reordered': np.uint16}

for (feat, dtype) in feat_dtype.items():
    product_feats[feat] = product_feats[feat].astype(dtype)

In [22]:
product_feats = pd.merge(product_feats, products, on='product_id')

In [25]:
# setting index
product_feats = product_feats.set_index('product_id')

In [26]:
product_feats.head()

,_prod_total_orders,_prod_total_users,_prod_total_reordered,_prod_avg_days_since_prior_order,_prod_rate_orders,_prod_rate_users,_prod_rate_reordered,aisle_id,department_id
product_id,,,,,,,,,
1,178,144,102,10.427711,0.000098,0.000726,0.573034,61,19
2,7,7,1,14.428572,0.000004,0.000035,0.142857,104,13
3,35,25,24,10.750000,0.000019,0.000126,0.685714,94,7
4,37,33,18,14.451612,0.000020,0.000166,0.486486,38,1
5,2,2,0,5.500000,0.000001,0.000010,0.000000,5,13


## Features by User X Product

[x]  Total number of orders<br>
[x]  Average days since prior order<br>
**[ ]  Rate of orders**<br>
**[ ]  Rate of reordered**

In [27]:
agg_func = {'order_id': ['nunique'],
            'days_since_prior_order': ['mean']}

user_product_feats = priors_orders.groupby(['user_id', 'product_id']).agg(agg_func)

In [28]:
# renaming columns
user_product_feats = user_product_feats[['days_since_prior_order', 'order_id']]

user_product_feats.columns = ['_uxp_avg_days_since_prior_order', '_uxp_total_orders']

In [29]:
user_product_feats.head()

_uxp_avg_days_since_prior_order  _uxp_total_orders
user_id product_id                                                    
1       196                                     0.0                  2
        12427                                  17.5                  2
        13176                                  15.0                  1
        46149                                  14.0                  1
2       9124                                    8.0                  1